In [1]:
from os import listdir
from pprint import pprint
from tqdm import tqdm

from typing import List

import pdfplumber
import pandas as pd

from model.columns_pdf import ColumnsPdf
from model.pdf import Pdf
from model.cpfl import Cpfl


In [5]:
from decimal import Decimal
import pandas as pd

class Cpfl:
    filename = ...
    pdf = ...

    def __init__(self, filename):
        self.filename = filename
        self.pdf = pdfplumber.open(self.filename)
        ...

    def __exit__(self):
        if self.pdf is not None:
            self.pdf.close()

    @property
    def view_pdf(self):
        im = self.pdf.pages[0].to_image(resolution=120)
        return im.debug_tablefinder({
            "vertical_strategy": "explicit", "horizontal_strategy": "explicit", "snap_tolerance": 3,
            "explicit_vertical_lines": [
                45, 60, 150, 175, 220, 255, 270, 305, 350, 390, 410, 440, 490, 520, 550], 
            "explicit_horizontal_lines": [290, 312, *[322+(10*i) for i in range(23)]],
            })

    @property
    def frame_table(self):
        extracted = self.pdf.pages[0].extract_tables({
            "vertical_strategy": "explicit", "horizontal_strategy": "explicit", "snap_tolerance": 3,
            "explicit_vertical_lines": 
            [
                45, 60, 150, 175, 220, 255, 270, 305, 350, 390, 410, 440, 490, 520, 550
            ], 
            "explicit_horizontal_lines": [290, 312, *[322+(10*i) for i in range(23)]],
        })[0]

        return pd.DataFrame(extracted[1:], columns=extracted[0])

In [6]:
cpfl = Cpfl("in/MAGSAC EMPRESA JUNHO 2.pdf")
cpfl.frame_table

,Cod.\n115,Descrição da Operação\nNº 902903429061,Mês\nRef.,Quant.\nRegistrada,Quant.\nFaturada,Unid.\nMed.,Tarifa com\nTributos R$,Valor Total\nOperação R$,Base Cálculo\nICMS R$,Aliq.\nICMS%,ICMS,Base Cálculo\nPIS/COFINS,"PIS\n1,14%","COFINS\n5,28%"
0,,Energia ACL,MAI/21,"573.675,199","573.675,199",,"0,51848728","297.443,29",,,,,,
1,,Total Distribuidora,,,,,,"297.443,29",,,,,,
2,,Desconto Energia ACL,,,,,,"136.496,36-",,,,,,
3,,Ajuste TUSD,FEV/21,,,,,"111,45",,,,,,
4,0999,Credito Subv Tarifa TUSD,,,,,,"23.734,71-",,,,,,
5,,Total a Pagar,,,,,,"137.323,67",,,,,,
6,,,,,,,,,,,,,,
7,,,,,,,,,,,,,,
8,,,,,,,,,,,,,,
9,,,,,,,,,,,,,,


In [ ]:
def read_pdf(path: str):
    with pdfplumber.open(path) as pdf:
        texts = []
        table = []
        for page in pdf.pages:
            extracted = page.extract_text(y_tolerance=4)
            if extracted is None:
                continue
            texts += filter(lambda i: i, extracted.split("\n"))
            table += page.extract_table(table_settings={
                "vertical_strategy": "lines",
                "horizontal_strategy": "text",
                "snap_tolerance": 2,
            }) or []
    return texts, table

# DEBUG
# texts, table = read_pdf("/home/marcelo-a/Downloads/Faturas/Varejo/Loja 08 - Elektro Junho 21.pdf") # Elektro
texts, table = read_pdf("/home/marcelo-a/Downloads/Faturas/Gestao/MAGSAC EMPRESA JUNHO 2.pdf") # Elektro
pd.DataFrame(texts)
# texts[17].split()[-2]
# texts[17].split()[-1]
# texts[60].split()[-2]
# texts[62].split()

In [ ]:
pd.DataFrame(texts1)[:30]

In [ ]:
def convert_table(texts: list, table: list):
    list_lines = []
    list_exists = []
    for line in table:
        for j, text in enumerate(texts):
            if line[0] in text:
                if j in list_exists:
                    continue
                else:
                    list_exists.append(j)
                
                columns_pdf = ColumnsPdf()
                columns_pdf.cci = text.split()[0] if text.split()[0].isdigit() else ""
                columns_pdf.descricao = line[0]
                columns_pdf.leitura_alterior = line[1]
                columns_pdf.leitura_atual = line[2]
                columns_pdf.registrado = line[3]
                columns_pdf.faturado = line[4]
                columns_pdf.tarifa_com_icms = line[5]
                columns_pdf.base_icms = line[6]
                columns_pdf.aliq_icms = line[7]
                columns_pdf.icms = line[8]
                columns_pdf.valor = line[9]
                columns_pdf.tarifa_sem_impostos = text.split()[-1] if line[-1] and text.split()[-1] != line[-1] else ""
                list_lines.append(columns_pdf)
                break
    columns_pdf = ColumnsPdf()
    columns_pdf.cci = texts[j+1].split()[0]
    columns_pdf.descricao = " ".join(texts[j+1].split()[1:-1])
    columns_pdf.valor = texts[j+1].split()[-1]
    list_lines.append(columns_pdf)
    return list_lines

# DEBUG
# convert_table(texts, table)

In [ ]:
def convert(texts: list, table: list):
    pdf = Pdf()

    numbers = texts[2].split(' ')
    pdf.n_instalacao, pdf.n_cliente, pdf.n_medidor = numbers[0], numbers[1], numbers[2]
    pdf.cnpj = texts[3].split('CPF/CNPJ:')[1].split(' ')[1]
    pdf.endereco = texts[6]
    pdf.subgrupo = texts[9].split()[1]
    pdf.data_emissao= " ".join(texts[-4].split()[:3])
    pdf.referente= " ".join(texts[-4].split()[3:5])
    pdf.vencimento= " ".join(texts[-4].split()[5:])
    if "Azul" in texts[12]:
        pdf.modalidade = "Azul"
        pdf.demanda_hora_ponta = texts[13].split()[1]
        pdf.demanda_hora_fora_ponta = texts[13].split()[4]
    elif "Verde" in texts[12]:
        pdf.modalidade = "Verde"
        pdf.demanda_unica = texts[13].split()[0]
    
    for text in texts:
        if 'CEP' in text:
            pdf.cep = text.split(' ')[1]
            break

    pdf.tabela = convert_table(texts, table)

    for line in pdf.tabela:
        if line.aliq_icms[:-1].isdigit() and line.aliq_icms[:-1] != "0" :
            pdf.aliquota = line.aliq_icms
            break
    else:
        pdf.aliquota = "0%"
            
    return pdf

# DEBUG
# TODO
# table.insert(33, ['PIS/PASEP (0,59%)', '', '', '', '', '', '0,00', '0%', '0,00', '77,21-']) # teste
# texts[0].insert(54, '0699 PIS/PASEP (0,59%) 0,00 0% 0,00 77,27-') # teste

In [ ]:
def full(path: str):
    texts, table = read_pdf(path)
    pdf = convert(texts, table)
    pdf.filename = path.split("/")[-1]
    return pdf

# DEBUG
# example = full("in/Barbosa ENEL_SP loja 28 mai21.pdf")
# example = full("in/Barbosa ENEL_SP loja 37 abr21.pdf")
# example.to_dataframe()
# example.tabela_to_dataframe().to_csv('t.csv')

In [ ]:
extracted = []
path = "in"
for file in tqdm(listdir(path)):
    try:
        extracted.append(full("/".join([path, file])))
    except Exception as e:
        print(file)
        raise e


In [ ]:
for text in tqdm(extracted):
    pd.concat([text.to_dataframe(), text.tabela_to_dataframe()]).to_csv('out/' + text.filename.replace('.pdf', '.csv'))

In [ ]:
# from wand.image import Image
# from os import listdir
# from tqdm import tqdm
# from IPython.display import display, clear_output

# for t in tqdm(extracted):
#     clear_output()
#     display(t.to_dataframe())
#     display(Image(filename="/".join([path, t.filename]), resolution=200))
#     display(t.tabela_to_dataframe())
#     input('Press enter to skip')
# # img

In [ ]:
from wand.image import Image

with open("in/Barbosa ENEL_SP loja 19 abr21.pdf", "r") as p:
    /help Image(file=p)

# TODO
- [x] Adicionar `Data Emissao`
- [x] Adicionar `Referente a`
- [x] Adicionar `Vencimento`
- [x] Adicionar `Subgrupo`
- [x] Adicionar `Modalidade`
- [x] Adicionar `Demanda Contratada` / `Hora Ponta` & `Hora Ponta Fora`
- [x] Adicionar `Aliquota (Ja está dentro da tabela)`